In [1]:
import json, os, sys, re
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from time import time
from tqdm import tqdm

from gem.utils import graph_util, plot_util
from gem.evaluation import visualize_embedding as viz
from gem.evaluation import evaluate_graph_reconstruction as GR

from gem.embedding.gf       import GraphFactorization     as GF
from gem.embedding.hope     import HOPE
from gem.embedding.lap      import LaplacianEigenmaps     as LAP
from gem.embedding.lle      import LocallyLinearEmbedding as LLE
from gem.embedding.node2vec import node2vec               as N2V

In [2]:
model = GF(d=32, max_iter=100000, eta=1*10**-4, regu=1.0)
# model = HOPE(d=32, beta=0.01)
# model = LAP(d=32)
# model = LLE(d=32)
# model = N2V(d=32, max_iter=1000, walk_len=80, num_walks=10, con_size=10, ret_p=1, inout_p=1)

In [3]:
# build graph
write_to_path = "ability_graph.txt"
build = True

if build:
    record = []
    root = "champ/en/"
    for f in tqdm(os.listdir(root)):
        df = pd.read_csv(root + f)
        for r in range(df.shape[0]):
            # print(df.iloc[r].tolist())
            vec = df.iloc[r].tolist()[2:]
            record.append(vec)
    record = np.array(record)
    n_abi, n_feature = record.shape
    print(n_abi, n_feature)
    np.save("all_champ_abilities.npy", record)
    
    with open(write_to_path, "w") as fout:
        for f in tqdm(range(n_feature)):
            arr = record[:, f]
            non = np.nonzero(arr)[0]
            L = len(non)
            
            for i in range(L - 1):
                for j in range(i + 1, L):
                    edge = str(non[i]) + "\t" + str(non[j]) + "\n"
                    fout.write(edge)
graph = graph_util.loadGraphFromEdgeListTxt(write_to_path, directed=False)
print("Num nodes: %d, num edges: %d" % (graph.number_of_nodes(), graph.number_of_edges()))

100%|███████████████████████████████████████████████████████████████████████████████| 159/159 [00:00<00:00, 268.83it/s]


811 46


100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [00:01<00:00, 33.50it/s]


Num nodes: 811, num edges: 276085


In [4]:
Y, t = model.learn_embedding(graph=graph, edge_f=None, is_weighted=True, no_python=True)

print(type(Y), Y.shape)
print(t)

np.save("embedding/ability_emb_gf_32d.npy", Y)

AttributeError: 'GraphFactorization' object has no attribute '_data_set'